In [1]:
from schellingmob import SchellingAgent, SchellingModel
import matplotlib.pyplot as plt
import measures_individual_schelling
import numpy as np
import skmob
import pandas as pd

import altair as alt
import math

In [2]:

side= 25
jump_plots =[]

def distance(p, q, type="euclid"):
    if type=="euclid":
        return math.sqrt((p[0] - q[0])**2 + (p[1] - q[1])**2)
    elif type=="moore":
        return max(abs(p[0] - q[0]), abs(p[1] - q[1]))
    elif type=="neumann":
        return abs(p[0] - q[0]) + abs(p[1] - q[1])
    
    
def centertown(pos, side):
    center = (side/2,side/2)
    radius= side /10
    return distance(pos,center) < radius+ side/10


In [3]:
texts = []
maps = []
trajs = []
jumps = []

mean_jumps = []

for i,ab in enumerate([(0,0), (0,-3),(3,0), (3,-3)]):
    
    city = SchellingModel(side=side, agents_report= True, seed=0,
                          mobility={"model":"gravity", "alpha":ab[0], "beta":ab[1]}, town= (ab[0] !=0))
    step=0
    
    while city.running and city.schedule.steps <500 :
        city.step()
        step+=1
        
    tot_seg = city.datacollector.get_model_vars_dataframe().iloc[-1]["total_segregation"]
    tot_seg = round(tot_seg,2)

    df = city.datacollector.get_agent_vars_dataframe().reset_index()
    df["x"] = df["pos"].apply(lambda x: x[0])
    df["y"] = df["pos"].apply(lambda x: x[1])
    agents = df.groupby("AgentID")["pos"].apply(set).reset_index(name="visited")
    agents["n_of_pla"] = agents["visited"].apply(lambda x: len(x))
    nome = agents.sort_values(by="n_of_pla").iloc[-1]["AgentID"]
    agents = df.groupby("AgentID")["pos"].apply(set).reset_index(name="visited")
    agents["travel"] = agents["visited"].apply(lambda x: len(x) > 1)
    df = df.merge(agents[["AgentID", "travel"]], on="AgentID", how="left")
    df = df[df["travel"]]
    tdf = skmob.TrajDataFrame(df, latitude='x', longitude='y', datetime='Step', user_id='AgentID')
    
    perc_empt_cent = (len([e for e in city.grid.empties if centertown(e,side)])/len([(x,y) for x in range(side) for y in range(side) if centertown((x,y),side)]))
    perc_empt_cent = round(perc_empt_cent, 2)
    
    model = ["Original (O)","Distance (D)","Relevance (R)","Gravity (G)"][i]
    
    text = alt.Chart({'values':[{}]}).mark_text(
    align="center", baseline="top", fontSize=18
    ).encode(
    x=alt.value(100),  # pixels from left
    y=alt.value(25),  # pixels from top
    text=alt.value([f"{model}",
                f"(α = {ab[0]}, β = {ab[1]})",    
                f"S = {tot_seg}, n = {step}"])).properties(width=200,
        height=100)
    
    
    texts.append(text)
    
    x, y = np.meshgrid(range(side), range(side))
    map_ = np.array([[city.grid.get_cell_list_contents((x,y))[0].type \
                   if len(city.grid.get_cell_list_contents((x,y)))>0 else 0 \
                   for x in range(side)] for y in range(side)])

    source = pd.DataFrame({'x': x.ravel(),'y': y.ravel(),
                         'z': map_.ravel()})

    map_ = alt.Chart(source).mark_rect().encode(
        alt.X('x:O', axis=None),
        alt.Y('y:O', axis=None),
        color=alt.Color('z:Q', legend=None).scale(domain = [0.9,1,2], range=["#000000", "red", "blue"])
    ).properties(  width=200,
        height=200)

    maps.append(map_)
    
    
   
    map_relevance = np.array([[city.cell_relevance[(x,y)] \
                   for x in range(side)] for y in range(side)])

    source = pd.DataFrame({'x': x.ravel(),
                         'y': y.ravel(),
                         'z': map_relevance.ravel()})
    
    if ab[0] == 0:
        color_relevance = alt.Color('z:Q', legend=None).scale(domain = [1], range=["#000000"])
    else:
        color_relevance = alt.Color('z:Q', legend=None).scale(domain = [map_relevance.min(),map_relevance.max()], range=["#000000", "#ffffff"])
    
    map_relevance = alt.Chart(source).mark_rect().encode(
        alt.X('x:O', axis=None),
        alt.Y('y:O', axis=None),
        color= color_relevance
    ).properties(  width=200,
        height=200)
    
    
    agentdf = city.datacollector.get_agent_vars_dataframe().reset_index()
    agentdf = agentdf[agentdf["AgentID"]==nome]
    agent_traj = agentdf["pos"]
    
    print(agent_traj.iloc[0], agent_traj.iloc[-1])

    df = pd.DataFrame({
    'x':[ p[0] for p in agent_traj],
    'y': [ p[1] for p in agent_traj]})

    arrows = alt.Chart(df[0:2]).mark_line().encode(
                        x= alt.X('x:O', axis=None),
                        y= alt.Y('y:O', axis=None),
                        color=alt.value('red'),
                        strokeWidth=alt.value(2)
                        ).properties(  width=200,
                                    height=200)


    for a in range(1,len(agent_traj)-2):

        arrow = alt.Chart(df[a:a+2]).mark_line().encode(
                            x= alt.X('x:O', axis=None),
                            y= alt.Y('y:O', axis=None),
                                color=alt.value('red'),
                            strokeWidth=alt.value(2)
                            ).properties(  width=200,
                                height=200)

        arrows = arrows + arrow

    trajs.append(map_relevance + arrows)

    
    
    jump_len = measures_individual_schelling.jump_lengths(tdf, show_progress=False)
    jump_list = []

    for j in jump_len["jump_lengths"]:
        jump_list += list(j)

    jump_list = pd.DataFrame(jump_list)
    jump_list[jump_list[0] > 0]
    data = jump_list[0]
    data = data[data != 0]
    
    data = pd.DataFrame({"jump": data})

    if i==0:        
        jump = alt.Chart(data).mark_bar( color='#abab').encode(
            alt.X("jump",title="jump lenght",  bin=alt.Bin(extent=[0, 35], step=5),  scale=alt.Scale(domain=[0, 35])),
            y= alt.Y('count()', title="# jumps"),
        ).properties(  width=180, 
            height=180)
    else:
        jump = alt.Chart(data).mark_bar( color='#abab').encode(
            alt.X("jump",title="jump lenght",  bin=alt.Bin(extent=[0, 35], step=5),  scale=alt.Scale(domain=[0, 35])),
            y= alt.Y('count()', title= None),
        ).properties(  width=180, 
            height=180)
        
    mean_jumps.append(np.mean(data)[0])
    jumps.append(jump)


(2, 13) (11, 3)


/home/dgambetta/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


(14, 13) (2, 5)


/home/dgambetta/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


(14, 4) (10, 22)


/home/dgambetta/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


(0, 7) (18, 6)


/home/dgambetta/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [4]:
data = alt.Data(values=[{'x': 'A'}])
a = (alt.Chart(data).mark_text(text='a)', size=15, x=0, y=-20, dx = 0, dy= 0))
b = (alt.Chart(data).mark_text(text='b)', size=15, x=0, y=-20, dx = 0, dy=0))
c = (alt.Chart(data).mark_text(text='c)',  size=15, x=0, y=-20, dx = 0, dy= 0))
d = (alt.Chart(data).mark_text(text='d)',  size=15, x=0, y=-20, dx = 0, dy= 0))

e = (alt.Chart(data).mark_text(text='e)',  size=15, x=0, y=-20, dx = 0, dy= 0))
f = (alt.Chart(data).mark_text(text='f)',  size=15, x=0, y=-20, dx = 0, dy= 0))
g = (alt.Chart(data).mark_text(text='g)',  size=15, x=0, y=-20, dx = 0, dy= 0))
h = (alt.Chart(data).mark_text(text='h)',  size=15, x=0, y=-20, dx = 0, dy= 0))

i = (alt.Chart(data).mark_text(text='i)',  size=15, x=0, y=-20, dx = 0, dy= 0))
j = (alt.Chart(data).mark_text(text='j)',  size=15, x=0, y=-20, dx = 0, dy= 0))
k = (alt.Chart(data).mark_text(text='k)',  size=15, x=0, y=-20, dx = 0, dy= 0))
l = (alt.Chart(data).mark_text(text='l)',  size=15, x=0, y=-20, dx = 0, dy= 0))

In [5]:
md0 = (alt.Chart(data).mark_text(text="d="+str(round(mean_jumps[0],2)),  size=20, x=0, y=0, dx = 130, dy= 15))
md1 = (alt.Chart(data).mark_text(text="d="+str(round(mean_jumps[1],2)),  size=20, x=0, y=0, dx = 130, dy= 15))
md2 = (alt.Chart(data).mark_text(text="d="+str(round(mean_jumps[2],2)),  size=20, x=0, y=0, dx = 130, dy= 15))
md3 = (alt.Chart(data).mark_text(text="d="+str(round(mean_jumps[3],2)),  size=20, x=0, y=0, dx = 130, dy= 15))
overline = (alt.Chart(data).mark_text(text="_",  size=20, x=0, y=0, dx = 100, dy= -8))

In [6]:
alt.hconcat(jumps[0]+i+md0 , jumps[1]+j+md1 , jumps[2]+md2 , jumps[3]+l+md3).resolve_scale(
    color='independent') 

alt.HConcatChart(...)

In [7]:
alt.vconcat(alt.hconcat(texts[0] , texts[1] , texts[2] , texts[3] ).resolve_scale(
    color='independent')  &
    
    alt.hconcat(maps[0]+a , maps[1]+ b , maps[2]+c , maps[3]+d ).resolve_scale(
    color='independent')  &

    alt.hconcat(trajs[0]+e , trajs[1]+f , trajs[2]+g , trajs[3]+h ).resolve_scale(
    color='independent')  &

    alt.hconcat(jumps[0]+i , jumps[1]+j , jumps[2]+k , jumps[3]+l).resolve_scale(
    color='independent') 
            
           ).resolve_scale(
    color='independent'
).configure_axis(
    gridOpacity=0.5,titleFontWeight="normal",  labelFontSize =16, titleFontSize=19
    
    )

alt.VConcatChart(...)